In [1]:
library(magick)
library(lubridate)

Linking to ImageMagick 7.1.1.2
Enabled features: cairo, fontconfig, freetype, fftw, rsvg, webp, x11
Disabled features: ghostscript, heic, lcms, pango, raw


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [2]:
path.reconyx <- "Reconyx"
folder.cannon <- "HW-resized-52"
list.files(path.reconyx)

# Each have some number of reconyx folders
path.build <- list.files(file.path(path.reconyx, "Build"), full.names = TRUE)
path.build.day <- list.files(file.path(path.reconyx, "Build-Day"), full.names = TRUE)
path.demo <- list.files(file.path(path.reconyx, "Demo"), full.names = TRUE)
path.cannon.orig <- list.files(file.path(path.reconyx, "HW"), full.names = TRUE)
path.cannon <- list.files(file.path(path.reconyx, folder.cannon), full.names = TRUE)

print(paste("Build:", length(list.files(path.build))))
print(paste("Build daytime:", length(list.files(path.build.day))))
print(paste("Demo:", length(list.files(path.demo))))
print(paste("Cannon:", paste0(folder.cannon, ":"), length(list.files(path.cannon))))

# # Note: can't filter files by time up here, because they don't keep their mtime when coming up into GCS
# x <- list.files(path.build, full.names = TRUE)
# x.info = file.info(list.files(path.build, full.names = TRUE))
# build.keep <- x.info %>% 
#   mutate(mtime_hour = hour(mtime)) %>% 
#   filter(between(mtime_hour, 8, 19))
# print(paste("Build daytime:", length(list.files(path.build))))

[1] "Build"         "Build-Day"     "Demo"          "HW"           
[5] "HW-del"        "HW-resized-40" "HW-resized-52" "HW-resized-60"

[1] "Build: 5027"
[1] "Build daytime: 2483"
[1] "Demo: 3252"
[1] "Cannon: HW-resized-52: 5102"


Define the function that reads and joins images, and writes the joined images to a video file.

This function uses video files because of https://docs.ropensci.org/magick/reference/video.html and https://github.com/ropensci/magick/issues/260

In [3]:
reconyx_animate <- function(path_imgs, file_out, fps = 20, tz = "America/Los_Angeles") {
    stopifnot(require(lubridate))
    # https://www.nagraj.net/notes/gifs-in-r/
    imgs <- list.files(path_imgs, full.names = TRUE)
    
    # # Hacky system to take every other image to deal with space/size issues
    # if (cannon) {
    #     imgs <- imgs[c(TRUE, FALSE)]
    #     message(paste("subset to ", length(imgs), images))
    # }
    
    message(paste("reading", as.character(lubridate::now(tz)), tz))
    img_list <- lapply(imgs, image_read)
    
    ## join the images together
    message(paste("joining", as.character(lubridate::now(tz)), tz))
    img_joined <- image_join(img_list)
    
    # # NOTE: could make a GIF here. Making video based on notes above
    # img_joined <- image_join(img_list)
    # image_write_gif(img_joined, path = file_out, delay = 1/fps)
    
    message(paste("writing mp4", as.character(lubridate::now(tz)), tz))
    image_write_video(img_joined, path = file_out, framerate = fps)
    message(paste("done writing mp4", as.character(lubridate::now(tz)), tz))
}

Now run the function on the things

In [ ]:
# reconyx_animate(path.cannon.orig, "CS-202324-build-slr-test.mp4", fps = 0.5)
reconyx_animate(path.cannon, "CS-202324-build-slr.mp4", fps = 40)
# reconyx_animate(path.build, "CS-202324-build.mp4", fps = 40)
# reconyx_animate(path.build.day, "CS-202324-build-day.mp4", fps = 40)
# reconyx_animate(path.demo, "CS-202324-demo.mp4", fps = 40)

reading 2024-03-23 07:53:58 America/Los_Angeles

joining 2024-03-23 08:01:28 America/Los_Angeles

writing mp4 2024-03-23 08:01:28 America/Los_Angeles

